<a href="https://colab.research.google.com/github/cjfghk5697/Pytorch_Lightning/blob/main/pytorch_lightning_Step_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOW TO ORGANIZE PYTORCH INTO LIGHTNING


In [10]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Move your Computational Code
Move the model architecture and forward pass to your LightningModule.

In [14]:
class LitModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.layer_1=nn.Linear(28*28,128)
    self.layer_2=nn.Linear(128,10)
  
  def forward(self, x):
    x=x.view(x.size(0),-1)
    x=self.layer_1(x)
    x=F.relu(x)
    x=self.layer_2(x)
    return x

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
    return [optimizer], [lr_scheduler]
  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = F.cross_entropy(y_hat, y)
    return loss
  def validation_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self(x)
      val_loss = F.cross_entropy(y_hat, y)
      self.log("val_loss", val_loss)
  def test_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self(x)
      test_loss = F.cross_entropy(y_hat, y)
      self.log("test_loss", test_loss)
  def predict_step(self, batch, batch_idx):
      x, y = batch
      pred = self(x)
      return pred


# 2. Move the Optimizer(s) and LR Scheduler(s)
Move your optimizers to the configure_optimizers() hook.

# 3. Configure the Training Logic
Lightning help training loop, manges all of the associated comonetns for you. For example, Using training_step() method need batch and batch index. Optionally, it can take optimizer_idx if your LightningModule defines multiple optimizers within its configure_optimizers() hook.

# 4. Configure the Validation Logic
You see Lightning help Training Logic for you. As you might expect Validation Logic also help you. When using Lightning, simply override the validation_step() method which takes the current batch and the batch_idx as arguments. Optionally, it can take dataloader_idx if you configure multiple dataloaders. To add an (optional) validation loop add logic to the validation_step() hook (make sure to use the hook parameters, batch and batch_idx in this case).

And you can run only the validation loop using validate() method.



In [ ]:
model = LitModel()
trainer.validate(model)

# 5. Congfigure Testing Logic
Lightning also help the testing llop for you. (test_step()method.) 

In [ ]:
model = LitModel()
trainer.test(model)

# 6. Configure Prediction Logic
Lightning help prediction Logic.(predict_step()).

# 7. Coda or to(Device) Calls
If you have any explicit calls to .cuda() or .to(device), you can remove them since Lightning makes sure that the data coming from DataLoader and all the Module instances initialized inside LightningModule.__init__ are moved to the respective devices automatically. If you still need to access the current device, you can use self.device anywhere in your LightningModule except in the __init__ and setup methods.

In [ ]:
class LitModel(LightningModule):
    def training_step(self, batch, batch_idx):
        z = torch.randn(4, 5, device=self.device)
        ...

Hint: If you are initializing a Tensor within the LightningModule.__init__ method and want it to be moved to the device automatically you should call register_buffer() to register it as a parameter.

In [ ]:
class LitModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.register_buffer("running_mean", torch.zeros(num_features))